In [5]:
import numpy as np
import pandas as pd
import lasagne
import theano
import theano.tensor as T
import random
import time
import connect_four as cccc
import sys
from numpy import random
#import seaborn
import matplotlib.pyplot as plt
%matplotlib inline


C:\Users\Todd\Anaconda2\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [6]:
#initialization

nets = []
play_nets = []
activation = lasagne.nonlinearities.tanh

hidden_units = 50
num_filters = 8
num_nets = 1
drop_rate = 0.1
num_play_nets = 10


for i in range(num_nets):
    value_in = lasagne.layers.InputLayer(shape=(None,42))
    
    value_shape = lasagne.layers.ReshapeLayer(value_in, (-1, 1, 6, 7))
    
    value_conv1 = lasagne.layers.Conv2DLayer(value_shape, num_filters = num_filters, filter_size = 4,
                                             W=lasagne.init.GlorotUniform(), nonlinearity = activation)
    
#   value_drop1 = lasagne.layers.DropoutLayer(value_conv1,p=drop_rate)
    
#     value_conv2 = lasagne.layers.Conv2DLayer(value_conv1, num_filters = num_filters, filter_size = 3,
#                                              W=lasagne.init.GlorotUniform(), nonlinearity = activation)
#   value_drop2 = lasagne.layers.DropoutLayer(value_conv2,p=drop_rate)
    
    value_hid1 = lasagne.layers.DenseLayer(value_in, num_units=hidden_units,W=lasagne.init.GlorotUniform(),
                                              nonlinearity = activation)

    value_drop1= lasagne.layers.DropoutLayer(value_hid1,p=drop_rate)
    
#     value_hid2= lasagne.layers.DenseLayer(value_drop1, num_units=hidden_units,W=lasagne.init.GlorotUniform(),
#                                               nonlinearity = activation)

#     value_drop2= lasagne.layers.DropoutLayer(value_hid2,p=drop_rate)
    
#     value_hid3= lasagne.layers.DenseLayer(value_drop2, num_units=hidden_units,W=lasagne.init.GlorotUniform(),
#                                               nonlinearity = activation)

#     value_drop3= lasagne.layers.DropoutLayer(value_hid3,p=drop_rate)
    
#     value_hid4= lasagne.layers.DenseLayer(value_drop3, num_units=hidden_units,W=lasagne.init.GlorotUniform(),
#                                               nonlinearity = activation)

#     value_drop4= lasagne.layers.DropoutLayer(value_hid4,p=drop_rate)
    

    value_out = lasagne.layers.DenseLayer(value_drop1,
                                      num_units=1, nonlinearity = activation)
    nets.append(value_out)

# value_in = lasagne.layers.InputLayer(shape=(None,42))

# value_hid = lasagne.layers.DenseLayer(value_in, num_units = 100, nonlinearity = activation)

# value_drop = lasagne.layers.DropoutLayer(value_hid,p=drop_rate)

# value_out = lasagne.layers.DenseLayer(value_drop, num_units = 1, nonlinearity = activation)
# nets.append(value_out)

for i in range(num_play_nets):
    
    play_in = lasagne.layers.InputLayer(shape=(None,42))
    
    play_shape = lasagne.layers.ReshapeLayer(play_in, (-1, 1, 6, 7))
    
    play_conv1 = lasagne.layers.Conv2DLayer(play_shape, num_filters = num_filters, filter_size = 3,
                                             W=lasagne.init.GlorotUniform(), nonlinearity = activation)
    
#   value_drop1 = lasagne.layers.DropoutLayer(value_conv1,p=drop_rate)
    
#     play_conv2 = lasagne.layers.Conv2DLayer(play_conv1, num_filters = num_filters, filter_size = 3,
#                                              W=lasagne.init.GlorotUniform(), nonlinearity = activation)
#   value_drop2 = lasagne.layers.DropoutLayer(value_conv2,p=drop_rate)
    
    play_hid1 = lasagne.layers.DenseLayer(play_conv1, num_units=hidden_units,W=lasagne.init.GlorotUniform(),
                                              nonlinearity = activation)
    
#     play_hid2= lasagne.layers.DenseLayer(play_hid1, num_units=hidden_units,W=lasagne.init.GlorotUniform(),
#                                               nonlinearity = activation)


    play_out = lasagne.layers.DenseLayer(play_hid1,
                                      num_units=1, nonlinearity = activation)
    play_nets.append(play_out)
    
    
    
    
    
    
    
    
train_in = lasagne.layers.InputLayer(shape=(None,42))

train_hid = lasagne.layers.DenseLayer(train_in, num_units = 100, nonlinearity = activation)

train_out = lasagne.layers.DenseLayer(train_hid, num_units = 1, nonlinearity = activation)

In [8]:
def value_move(board,active_turn,output_funs,exploration = 0,noise = 0):
    X_sym = theano.tensor.matrix()
    y_sym = theano.tensor.ivector()
    dummy_board = np.copy(board)
    options = cccc.available_moves(dummy_board)
    
    
    if exploration > random.random():
        move = random.choice(options) 
    else:
        moves = np.zeros(7)
        for output_fun in output_funs:
            move_values = np.zeros(7)
            for choice in options:
                dummy_board = active_turn * board[:]
                dummy_board[np.where(dummy_board[:,choice]==0)[0][-1],choice] = 1
                reshapeable = np.copy(dummy_board)
                move_values[choice] = -1 * output_fun(-1* reshapeable.reshape(1,42))+ noise * np.random.random()
            available_move_values = np.array([move_values[move] for move in options])

            move = options[available_move_values.argmax(-1)]
            moves[move] += 1
        
        move = np.argmax(moves)
    return move + 1


class nn_ai:
    
    def __init__(self,output_fun, net = 'value',exploration = 0,noise = 0):
        self.output_fun = output_fun
        self.exploration = exploration
        self.noise = noise
        self.net = net
    
    def make_move(self,board,active_turn):
#         if self.net == 'policy':
#             move = policy_move(board,active_turn,self.output_fun,self.exploration)
        if self.net == 'value':
            move = value_move(board,active_turn,self.output_fun,self.exploration,self.noise)
        return move
    
def alpha_beta_move(board,active_turn,depth,alpha = 2):
    swap_dict = {1:-1,-1:1}
    dummy_board = np.zeros((6,7))
    dummy_board[:] = board[:]
    options = cccc.available_moves(board)
    random.shuffle(options)
    if len(options) == 1:
        dummy_board[np.where(dummy_board[:,options[0]]==0)[0][-1],options[0]] = active_turn
        if cccc.winner(dummy_board):
            return (1,options[0]+1)
        else:
            return (0,options[0]+1)
    if depth ==0:
        return (0, options[np.random.randint(len(options))]+1)

    best_value = -2
    candidate_move = None
    for x in options:
        height = np.where(dummy_board[:,x]==0)[0][-1]
        dummy_board[height,x] = active_turn
        if cccc.winner(dummy_board):
            return (1, x+1)
        (opp_value,opp_move) = alpha_beta_move(dummy_board,swap_dict[active_turn],depth-1,-best_value)
        if -opp_value > best_value:
            candidate_move = x+1
            best_value = -opp_value
        if -opp_value >= alpha:
            #print (options, x, best_value, alpha)
            break
        dummy_board[height,x] = 0

    return (best_value, candidate_move)

class alpha_beta:
    def __init__(self,depth):
        self.depth = depth
    def make_move(self,board,active_turn):
        #print (board,active_turn,self.depth)
        return alpha_beta_move(board,active_turn,self.depth)[1]

def get_max_future(future_board,value_fun):
    options = cccc.available_moves(future_board)
    dummy_board = np.copy(future_board)
    move_values = np.zeros(7)
    for move in options:
        dummy_board = np.copy(future_board)
        dummy_board[np.where(dummy_board[:,move]==0)[0][-1],move] = -1
        # dummy_board = dummy_board.reshape(1,42)
        if cccc.winner(dummy_board):
            move_values[move] = cccc.winner(dummy_board)
        else:
            reshapable = np.copy(dummy_board)
            reshapable = reshapable.reshape(1,42)
            move_values[move] = value_fun(reshapable)
    
    available_move_values = np.array([move_values[move] for move in options])
    dummy_board = np.copy(future_board)
    options_index = np.argmin(available_move_values)
    dummy_board[np.where(dummy_board[:,options[options_index]]==0)[0][-1],options[options_index]] = -1
    return np.amin(available_move_values), dummy_board

def play_one_move(current_board,value_funs,exploration = 0):
    options = cccc.available_moves(current_board)
    dummy_board = np.copy(current_board)
    move_values = np.zeros(7)
    values = []
    if np.random.random() < exploration:
        chosen_move = random.choice(options)
        dummy_board[np.where(dummy_board[:,chosen_move]==0)[0][-1],chosen_move] = 1
        explore = True
        if cccc.winner(dummy_board):
            value = cccc.winner(dummy_board)
        else:
            reshapable = np.copy(dummy_board)
            reshapable = reshapable.reshape(1,42)
            for value_fun in value_funs:
                values.append(-1 * value_fun(-1 * reshapable))
            value = np.mean(np.array(values))
    else:
        moves = np.zeros(7)
        explore = False
        for value_fun in value_funs:
            for move in options:
                dummy_board = np.copy(current_board)
                dummy_board[np.where(dummy_board[:,move]==0)[0][-1],move] = 1
                # dummy_board = dummy_board.reshape(1,42)
                if cccc.winner(dummy_board):
                    move_values[move] = cccc.winner(dummy_board)
                else:
                    reshapable = np.copy(dummy_board)
                    reshapable = reshapable.reshape(1,42)
                    move_values[move] = -1 * value_fun(-1 * reshapable)
            
            available_move_values = np.array([move_values[move] for move in options])
            options_index = np.argmax(available_move_values)
            move = options[options_index]
            moves[move] += 1
            values.append(np.amax(available_move_values))
    
        value = np.mean(np.array(values))
        move = np.argmax(moves)
        dummy_board = np.copy(current_board)
        dummy_board[np.where(dummy_board[:,move]==0)[0][-1],move] = 1
        

    return value , dummy_board , explore

def get_value(current_board,value_funs):
    values = []
    reshapable = np.copy(current_board)
    reshapable = reshapable.reshape(1,42)
    for value_fun in value_funs:
        values.append(value_fun(reshapable))
    return np.mean(np.array(values))

def get_inputs(log):
    boards = []
    piece = 1
    board = np.zeros((6,7))
    boards.append(np.copy(board))
    for move in log:
        board[np.where(board[:,move-1]==0)[0][-1],move-1] = piece
        piece = -piece
        boards.append(np.copy(board))
    return boards


def game_over(board):
    return cccc.winner(board) or cccc.is_full(board)

In [ ]:
# load the chromes
test = np.load('TD_cccc_100_1_8_million.npz')['arr_0']
lasagne.layers.set_all_param_values(train_out,test)


In [ ]:
test = np.load('TD_cccc_cnn_1_5_million(backup100).npz')['arr_0']
lasagne.layers.set_all_param_values(nets[0],test)

In [ ]:
for net in play_nets:
    lasagne.layers.set_all_param_values(net,test)

In [ ]:
for item in test:
    print item.shape

# Reinforcement Learning

In [9]:
X_sym = T.matrix()
y_sym = T.matrix()

input_history = []
output_history = []
results = {}
move_history = []
outputs = [lasagne.layers.get_output(output_layer, X_sym) for output_layer in nets]
outputs_det = [lasagne.layers.get_output(output_layer, X_sym,deterministic=True) for output_layer in nets]
train_output = lasagne.layers.get_output(train_out, X_sym)
play_outputs = [lasagne.layers.get_output(play_out, X_sym) for play_out in play_nets]

value_funs = [theano.function([X_sym],output) for output in outputs]
value_funs_det = [theano.function([X_sym],output_det) for output_det in outputs_det]
play_funs = [theano.function([X_sym],play_output) for play_output in play_outputs]
#training_funs = [theano.function([X_sym], train_output)]
training_funs = value_funs
params = [lasagne.layers.get_all_params(output_layer) for output_layer in nets]
objective_list = [T.mean(lasagne.objectives.squared_error(output,y_sym)) for output in outputs]

grads = [T.grad(objective_list[i],params[i]) for i in range(num_nets)]
self_play = 0

#flush training sets
input_history = []
output_history = []
results = {}
move_history = []

In [10]:
BATCH_SIZE = 256
batches_per_step = 50
training_per_step =0
train_duration = 10000
exploration = 0.25
exploration_min = 0.10
exploration_step = 0.995
future_discount = 0.5
minimax_str = 0
validation_str = 2
monte_carlo_duration = 0
print_freq = batches_per_step-1
valid_freq = 10
learning_speed = 0.005
updates = [lasagne.updates.nesterov_momentum(grads[i], params[i], learning_rate=learning_speed,momentum = 0.9)
           for i in range(num_nets)]
#updates = [lasagne.updates.sgd(grads[i], params[i], learning_rate=learning_speed) for i in range(num_nets)]

#updates = [lasagne.updates.rmsprop(grads[i],params[i], learning_rate = 1) for i in range(num_nets)]

# updates = [lasagne.updates.adam(grads[i], params[i], learning_rate = learning_speed) for i in range(num_nets)]
train_nets = [theano.function([X_sym, y_sym], objective_list[i], updates=updates[i]) for i in range(num_nets)]
temporal_index = 0.8
learning_index = 0.2
objective_total = 1000.0
noise = 0.5

In [12]:
param_values = lasagne.layers.get_all_param_values(value_out)
training_size = 5 * sum([param_values[i].size for i,_ in enumerate(param_values)])
for item in param_values:
    print item.shape
    print item.size
    
print training_size

(42L, 50L)
2100
(50L,)
50
(50L, 1L)
50
(1L,)
1
11005


In [13]:
training_size = 5 * sum([param_values[i].size for i,_ in enumerate(param_values)])
first_pass = True
training_data = [0]*training_size
target_data = [0]*training_size
current_write_index = 0
current_net_index = 0
epoch_objective_list = []
#first_pass = False

In [ ]:
t0 = time.clock()


param_values = lasagne.layers.get_all_param_values(value_out)
training_size = 5 * sum([param_values[i].size for i,_ in enumerate(param_values)])



# start training

data_generation = 0

for epoch in range(train_duration):
    
    t1 = time.clock()
    exploration = (exploration-exploration_min)* exploration_step + exploration_min
    if current_write_index >= training_size:
        data_generation +=1
        print '\n'
        print 'now starting generation {} of data'.format(data_generation)
        print '\n'
    current_write_index = current_write_index%training_size
    
    # continue to update training set
#     for _ in range(training_size/50):
    counter = 0
    player = 1
#     while(counter < training_size/20 or (first_pass and counter < training_size)):
    
#         current_state = cccc.empty_board()
        
#         self_play +=1
#         player_1 = value_funs_det[0]
#         player_2 = random.choice(play_funs)
#         if player == -1:
#             dummy = player_1
#             player_1 = player_2
#             player_2 = dummy

#         play_result = cccc.play(nn_ai([player_1],noise=noise),nn_ai([player_2],noise = noise))
#         boards = get_inputs(play_result.log)
            
#         learning_selection = random.choice(range(len(boards)))

#         # current player should be 1 if even -1 if odd
#         if learning_selection % 2 ==0:
#             learning_player = 1
#         else:
#             learning_player = -1
        
#         boards = [board*learning_player for board in boards]
#         learning_board = np.copy(boards[learning_selection] )
#         learning_pool = boards[learning_selection:]
#         if len(learning_pool)%2 == 0:
#             current_player = -1
#         else:
#             current_player = 1
            
#         future_board = learning_pool.pop(-1)*current_player
#         learning_target = cccc.winner(future_board)
#         current_player *=-1

#         while(learning_pool):
#             future_board = learning_pool.pop(-1).reshape(1,42) * current_player
#             learning_target = -1 * temporal_index * learning_target + (1-temporal_index)*training_funs[0](
#                 future_board)
#             current_player *= -1
#         player *= -1
#         training_data[current_write_index%training_size] = learning_board
#         target_data[current_write_index%training_size] = learning_target   
#         current_write_index +=1
#         counter +=1
        
#         ###############################################################
    
    while(counter < training_size/20 or (first_pass and counter < training_size)):
        player_1 = value_funs_det[0]
        player_2 = value_funs_det[0]
        play_result = cccc.play(nn_ai([player_1],noise=noise),nn_ai([player_2],noise = noise))
        boards = get_inputs(play_result.log)
        learning_selection = random.choice(range(len(boards)))
        
        if learning_selection % 2 ==0:
            learning_player = 1
        else:
            learning_player = -1
        self_play +=1
        learning_pool = []
        value_history = []
        
        boards = [board*learning_player for board in boards]
        learning_board = np.copy(boards[learning_selection])
        
        current_state = learning_board
        while not game_over(np.copy(current_state)):
             
            next_value , next_state , explore = play_one_move(current_state,value_funs)
            learning_pool.append(current_state)
            value_history.append(get_value(current_state,training_funs))
            current_state = -1 * np.copy(next_state)


        learning_pool.append(current_state)
        value_history.append(cccc.winner(current_state))

        
        future_board = learning_pool.pop(-1)
        learning_target = value_history.pop(-1)
        while(value_history):
            learning_target = -1 * temporal_index * learning_target + (1-temporal_index)*value_history.pop(-1)
        training_data[current_write_index%training_size] = learning_board
        target_data[current_write_index%training_size] = learning_target   
        current_write_index +=1
        counter +=1
           

        
        if counter%100 ==0:
            if first_pass:
                sys.stdout.write('\r writing training data {:5d} of {:5d}'.format(counter, training_size))
                sys.stdout.flush()
            else:
                sys.stdout.write('\r writing training data {:5d} of {:5d}'.format(counter, training_size/20))
                sys.stdout.flush()            
    print '\n'
    objective_totals = []
    first_pass = False
    current_net = 0
    for train_net in train_nets:
        current_net +=1
        for j in range(batches_per_step):
            t2=time.clock()
            targets = np.zeros((BATCH_SIZE,1))
            training = np.zeros((BATCH_SIZE,42))
            index_pool = range(training_size)
            random.shuffle(index_pool)
            objectives = []        

            while len(index_pool) > BATCH_SIZE:
                sample_indices = [index_pool.pop() for _ in xrange(BATCH_SIZE)]
                #Should try to use generators


                for k in range(BATCH_SIZE):
                    #train it on output_history evaluated by the opponent
                    training[k] = training_data[sample_indices[k]].reshape(1,42)
                    targets[k] = target_data[sample_indices[k]]



                objectives.append(train_net(training,targets))
                #break to only do 1 batch per run.
            objective_total = np.mean(objectives)
            

            if j % print_freq ==0 or j == (print_freq*4 ) / 5:
                print (('Net {}, Epoch {:5d}, Pass number {}, objective: {:0.5f}, exploration: {:0.2f}, '+
                       'number of games played: {}, step duration: {:1.3f}s').format(current_net,
                       epoch,j+1,float(objective_total),exploration,self_play,time.clock()-t2))


            epoch_objective_list.append([epoch,current_net, j+1,objective_total, self_play])
        print ' '
        print ' '
        test_result = {'wins':0,'ties':0,'losses':0}
        for j in range(100):
#             sys.stdout.write('\r playing validation game {:4d} of {:4d}'.format(j+1, 1000))
#             sys.stdout.flush()
            result = cccc.play(nn_ai(value_funs_det,'value'),alpha_beta(validation_str))
            if result.winner ==1:
                test_result['wins'] +=1
            if result.winner == 0:
                test_result['ties'] +=1
            if result.winner == -1:
                test_result['losses'] +=1

#         print '\n'
        print 'As X, neural network has a score of {:3d}-{:3d}-{:3d} vs {}-depth minimax'.format(test_result['wins'],
                                                               test_result['ties'],test_result['losses'],validation_str)

        test_result = {'wins':0,'ties':0,'losses':0}

        for j in range(100):
#             sys.stdout.write('\r playing validation game {:4d} of {:4d}'.format(j+1, 1000))
#             sys.stdout.flush()
            result = cccc.play(alpha_beta(validation_str),nn_ai(value_funs_det,'value'))
            if result.winner ==1:
                test_result['losses'] +=1
            if result.winner == 0:
                test_result['ties'] +=1
            if result.winner == -1:
                test_result['wins'] +=1
#         print '\n'
        print 'As O, neural network has a score of {:3d}-{:3d}-{:3d} vs {}-depth minimax'.format(test_result['wins'],
                                                            test_result['ties'],test_result['losses'],validation_str)
        print ' '
        print 'elapsed time: {:3.3f}s'.format(time.clock()-t0)
        print ' '
        # save the chromes

    print 'epoch duration {:2.2f}s'.format(time.clock()-t1)
    if epoch%valid_freq==0:

        np.savez('TD_cccc_last',lasagne.layers.get_all_param_values(value_out))
    if epoch%10==0:
        
#        lasagne.layers.set_all_param_values(play_nets[current_net_index%num_play_nets],lasagne.layers.get_all_param_values(
#            nets[0]))
        print 'play net {} updated'.format(current_net_index%num_play_nets)
        current_net_index +=1
    if epoch%100==0:
        np.savez('TD_cccc_last_100',lasagne.layers.get_all_param_values(nets[0]))


print('\n')

#for epoch in range(10):

        
#    t_epoch = time.clock()

#    result_list = [tourney(output) for i in range(BATCH_SIZE)]
#    fitness_list = [fitness(result) for result in result_list]
    
#    score= f_train(fitness_list)
       
#    t1=time.clock()-t_epoch
    
#    print('Epoch {}, duration {:.01f} seconds'.format(
#            epoch+1, t1))
#    print('Record is: {}-{}-{} with a score of {}'.format(sum([result['wins'] for result in result_list]),
#                                                          sum([result['ties'] for result in result_list]),
#                                                          sum([result['losses'] for result in result_list]),
#                                                          sum([fitness for fitness in fitness_list])))
#    print('mean score is {}'.format(score))
print('total time for neural network is {:.01f} seconds'.format(time.clock()-t0))


 writing training data   500 of 11005

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

f = open('supervised_connect_four.txt','r')
plot_list = f.read().splitlines()
plot_list = np.array([[float(x) for x in item[1:-1].split(',')] for item in plot_list])

f = open('supervised_tictactoe.txt','r')
plot_list2 = f.read().splitlines()
plot_list2 = np.array([[float(x) for x in item[1:-1].split(',')] for item in plot_list2])

In [ ]:
import seaborn
epoch_objective_list = np.array(epoch_objective_list)

plt.figure(figsize=(16,9))
plt.plot(50*epoch_objective_list[:,0]+epoch_objective_list[:,2],epoch_objective_list[:,3],color = 'blue',linewidth=3)
# plt.plot(plot_list[:,0],plot_list[:,2],color = 'green')
# plt.plot(50*plot_list2[:,0]+plot_list2[:,1]-900,plot_list2[:,2],color = 'red')
plt.title('Reinforcement Learning for Connect Four',fontsize=32)
plt.xlabel('Epoch',fontsize=24)
plt.ylabel('Mean Squared Error',fontsize = 24)
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)
ax = plt.subplot(111)    
ax.spines["top"].set_visible(False)    
# ax.spines["bottom"].set_visible(False)    
ax.spines["right"].set_visible(False)    
# ax.spines["left"].set_visible(False)    
  
# Ensure that the axis ticks only show up on the bottom and left of the plot.    
# Ticks on the right and top of the plot are generally unnecessary chartjunk.    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()
# plt.text(410, 0.61,"Supervised learning with connect four - loss",fontsize=20,color='blue')
# plt.text(410, 0.56,"Supervised learning with connect four - validation",fontsize=20,color='green')
#plt.text(410, 0.51," learning with with connect four - mean squared error",fontsize=20,color='blue')
plt.savefig('reinforcement_c4.png')
plt.show

In [ ]:
np.array(epoch_objective_list)[:,0]

In [ ]:
# save the chromes
np.savez('TD_cccc_cnn_adam_275_thousand',lasagne.layers.get_all_param_values(value_out))


In [ ]:
validation_str = 2
t0 = time.clock()
this_one = [theano.function([X_sym], train_output)]
player_1 = nn_ai(value_funs_det)
player_2 = alpha_beta(validation_str)
print ' '
print ' '
test_result = {'wins':0,'ties':0,'losses':0}
for j in range(100):
    result = cccc.play(player_1,player_2)
    if result.winner ==1:
        test_result['wins'] +=1
    if result.winner == 0:
        test_result['ties'] +=1
    if result.winner == -1:
        test_result['losses'] +=1


print 'As X, neural network has a score of {:3d}-{:3d}-{:3d} vs {}-depth minimax'.format(test_result['wins'],
                                                       test_result['ties'],test_result['losses'],validation_str)

test_result = {'wins':0,'ties':0,'losses':0}

for j in range(100):
    result = cccc.play(player_2,player_1)
    if result.winner ==1:
        test_result['losses'] +=1
    if result.winner == 0:
        test_result['ties'] +=1
    if result.winner == -1:
        test_result['wins'] +=1

print 'As O, neural network has a score of {:3d}-{:3d}-{:3d} vs {}-depth minimax'.format(test_result['wins'],
                                                    test_result['ties'],test_result['losses'],validation_str)
print ' '
print 'elapsed time: {:3.3f}s'.format(time.clock()-t0)
print ' '

In [ ]:
result = cccc.play(cccc.player(),nn_ai(value_funs_det))

In [ ]:
get_inputs(result.log)
print result.log

In [ ]:
for i in range(100):
    plt.imshow(training_data[i],interpolation = 'nearest')
    plt.show()
    print target_data[i]
training_data[0]

In [ ]:
if player == 1:
    learning_pool = [boards[i] for i in range(len(boards)) if i%2==0]
else:
    learning_pool = [boards[i] for i in range(len(boards)) if i%2==1]
print player
print learning_pool


In [ ]:
f = open('TD_connect_four.txt','w')

for item in epoch_objective_list:
    print>>f , item

In [ ]:
for i in range(10):
    print training_data[i],target_data[i],value_funs[0](training_data[i].reshape(1,42))

# Supervised Learning

In [ ]:
raw_training = pd.read_csv('connect-4.csv',header = None)

In [ ]:
training_set = []
targets_set = []
for index,row in raw_training.iterrows():
    input_dict = {'b':0,'x':1,'o':-1}
    game_state_dict = {'win':1,'draw':0,'loss':-1}
    game_state = game_state_dict[row[42]]
    row = row[:-1].apply(lambda x: input_dict[x])
    board = np.flipud(np.reshape(np.array(row),(6,7),order = 'F'))
    if np.sum(board)==1:
        board = -1 * np.copy(board)
    
    training_set.append(np.copy(board).reshape(1,42))
    targets_set.append(game_state)

dummy_list = list(zip(training_set,targets_set))
random.shuffle(dummy_list)
training_set,targets_set = zip(*dummy_list)

val_training_set = training_set[:len(training_set)/2]
val_targets_set = targets_set[:len(targets_set)/2]
training_set = training_set[len(training_set)/2:]
targets_set = targets_set[len(targets_set)/2:]

In [ ]:
X_sym = T.matrix()
y_sym = T.matrix()

output = lasagne.layers.get_output(value_out,X_sym)
output_det = lasagne.layers.get_output(value_out,X_sym,deterministic=True)
value_fun = theano.function([X_sym],output)
value_fun_det = theano.function([X_sym],output_det)
params = lasagne.layers.get_all_params(value_out)
objective = T.mean(lasagne.objectives.squared_error(output,y_sym))
grad = T.grad(objective,params)



In [ ]:
BATCH_SIZE = 256
train_duration = 500
minimax_str = 0
validation_str = 2
valid_freq = 100
learning_speed = 0.01
updates = lasagne.updates.nesterov_momentum(grad, params, learning_rate=learning_speed,momentum = 0.9)
#updates = lasagne.updates.sgd(grad, params, learning_rate=learning_speed)
train_net = theano.function([X_sym, y_sym], objective, updates=updates)
val_net = theano.function([X_sym, y_sym], objective)

In [ ]:
t0 = time.clock()

epoch_objective_list = []

for epoch in range(train_duration):


    t2=time.clock()

    objectives = []
    index_pool = range(len(targets_set))
    random.shuffle(index_pool)
    while len(index_pool) > BATCH_SIZE:
        
        sample_indices = [index_pool.pop() for _ in xrange(BATCH_SIZE)]

        targets = np.zeros((BATCH_SIZE,1))
        training = np.zeros((BATCH_SIZE,42))
        for k in range(BATCH_SIZE):
            training[k] = np.copy(training_set[sample_indices[k]])
            targets[k] = targets_set[sample_indices[k]]

        objectives.append(train_net(training,targets))
        #break to only do 1 batch per run.
    objective_total = np.mean(objectives)


    objectives = []
    index_pool = range(len(val_targets_set))
    random.shuffle(index_pool)
    while len(index_pool) > BATCH_SIZE:
        
        sample_indices = [index_pool.pop() for _ in xrange(BATCH_SIZE)]

        targets = np.zeros((BATCH_SIZE,1))
        training = np.zeros((BATCH_SIZE,42))
        for k in range(BATCH_SIZE):
            training[k] = np.copy(training_set[sample_indices[k]])
            targets[k] = targets_set[sample_indices[k]]

        objectives.append(val_net(training,targets))
        #break to only do 1 batch per run.
    val_total = np.mean(objectives)

    print (('Epoch {:5d},objective: {:0.5f}, validation: {:0.5f}, step duration: {:1.3f}s').format(
            epoch,float(objective_total),float(val_total),time.clock()-t2))
    
        
    
    
    
    epoch_objective_list.append([epoch,objective_total,val_total])

    # print 'epoch duration {:2.2f}s'.format(time.clock()-t2)
    if epoch%valid_freq==0:
        print ' '
        print ' '
        test_result = {'wins':0,'ties':0,'losses':0}
        for j in range(100):
            result = cccc.play(nn_ai(value_fun_det,'value'),alpha_beta(validation_str))
            if result.winner ==1:
                test_result['wins'] +=1
            if result.winner == 0:
                test_result['ties'] +=1
            if result.winner == -1:
                test_result['losses'] +=1


        print 'As X, neural network has a score of {:3d}-{:3d}-{:3d} vs {}-depth minimax'.format(test_result['wins'],
                                                               test_result['ties'],test_result['losses'],validation_str)

        test_result = {'wins':0,'ties':0,'losses':0}

        for j in range(100):
            result = cccc.play(alpha_beta(validation_str),nn_ai(value_fun_det,'value'))
            if result.winner ==1:
                test_result['losses'] +=1
            if result.winner == 0:
                test_result['ties'] +=1
            if result.winner == -1:
                test_result['wins'] +=1

        print 'As O, neural network has a score of {:3d}-{:3d}-{:3d} vs {}-depth minimax'.format(test_result['wins'],
                                                            test_result['ties'],test_result['losses'],validation_str)
        print ' '
        print 'elapsed time: {:3.3f}s'.format(time.clock()-t0)
        print ' '
        
        


print('\n')

#for epoch in range(10):

        
#    t_epoch = time.clock()

#    result_list = [tourney(output) for i in range(BATCH_SIZE)]
#    fitness_list = [fitness(result) for result in result_list]
    
#    score= f_train(fitness_list)
       
#    t1=time.clock()-t_epoch
    
#    print('Epoch {}, duration {:.01f} seconds'.format(
#            epoch+1, t1))
#    print('Record is: {}-{}-{} with a score of {}'.format(sum([result['wins'] for result in result_list]),
#                                                          sum([result['ties'] for result in result_list]),
#                                                          sum([result['losses'] for result in result_list]),
#                                                          sum([fitness for fitness in fitness_list])))
#    print('mean score is {}'.format(score))
print('total time for neural network is {:.01f} seconds'.format(time.clock()-t0))


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

f = open('supervised_connect_four.txt','r')
plot_list = f.read().splitlines()
plot_list = np.array([[float(x) for x in item[1:-1].split(',')] for item in plot_list])

f = open('supervised_tictactoe.txt','r')
plot_list2 = f.read().splitlines()
plot_list2 = np.array([[float(x) for x in item[1:-1].split(',')] for item in plot_list2])

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(plot_list[:,0],plot_list[:,1],color = 'blue',linewidth=3)
plt.plot(plot_list[:,0],plot_list[:,2],color = 'green')
plt.plot(50*plot_list2[:,0]+plot_list2[:,1]-900,plot_list2[:,2],color = 'red')
plt.title('Deep Learning',fontsize=32)
plt.xlabel('Epoch',fontsize=24)
plt.ylabel('Mean Squared Error',fontsize = 24)
plt.xticks(fontsize = 20)
plt.yticks(fontsize = 20)
ax = plt.subplot(111)    
ax.spines["top"].set_visible(False)    
# ax.spines["bottom"].set_visible(False)    
ax.spines["right"].set_visible(False)    
# ax.spines["left"].set_visible(False)    
  
# Ensure that the axis ticks only show up on the bottom and left of the plot.    
# Ticks on the right and top of the plot are generally unnecessary chartjunk.    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()
plt.text(410, 0.61,"Supervised learning with connect four - loss",fontsize=20,color='blue')
plt.text(410, 0.56,"Supervised learning with connect four - validation",fontsize=20,color='green')
plt.text(410, 0.51,"Reinforcement learning with tic tac toe - loss",fontsize=20,color='red')
plt.savefig('supervised_c4.png')
plt.show

In [ ]:
f = open('TD_connect_four.txt','r')
plot_list = f.read().splitlines()
plot_list = np.array([[float(x) for x in item[1:-1].split(',')] for item in plot_list])

In [ ]:
plt.plot(500*plot_list[:,0]+plot_list[:,1],plot_list[:,2])
plt.show

In [ ]:
f = open('supervised_tictactoe.txt','r')
plot_list = f.read().splitlines()
plot_list = np.array([[float(x) for x in item[1:-1].split(',')] for item in plot_list])

In [ ]:
plt.plot(50*plot_list[:,0]+plot_list[:,1],plot_list[:,2],color = 'red')
plt.title('Reinforcement Learning for Tic Tac Toe',fontsize=16)
plt.xlabel('Epoch',fontsize=16)
plt.ylabel('Mean Squared Error',fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
ax = plt.subplot(111)    
ax.spines["top"].set_visible(False)    
# ax.spines["bottom"].set_visible(False)    
ax.spines["right"].set_visible(False)    
# ax.spines["left"].set_visible(False)    
ax.get_xaxis().tick_bottom()    
ax.get_yaxis().tick_left()
plt.savefig('reinforcement_ttt.png')
plt.show
  

In [244]:
def alpha_beta_move(board,active_turn,depth,evaluation = lambda x: 0,alpha = 2):
    swap_dict = {1:-1,-1:1}
    dummy_board = np.copy(board)
    dummy_board = dummy_board.reshape((6,7))
    options = cccc.available_moves(dummy_board)
    random.shuffle(options)
    if len(options) == 1:
        dummy_board[np.where(dummy_board[:,options[0]]==0)[0][-1],options[0]] = active_turn
        if cccc.winner(dummy_board):
            return (1,options[0]+1)
        else:
            return (0,options[0]+1)
    if depth ==0:
        best_value = -2
        for x in options:
            height = np.where(dummy_board[:,x]==0)[0][-1]
            dummy_board[height,x] = active_turn
            eval_board = evaluation(dummy_board*active_turn)
            if  eval_board > best_value:
                best_value = eval_board
                candidate_move = x + 1
            dummy_board[height,x] = 0
        return (best_value, candidate_move)

    best_value = -2
    candidate_move = None
    for x in options:
        height = np.where(dummy_board[:,x]==0)[0][-1]
        dummy_board[height,x] = active_turn
        if cccc.winner(dummy_board):
            return (1, x+1)
        (opp_value,opp_move) = alpha_beta_move(dummy_board,swap_dict[active_turn],depth-1,evaluation,-best_value)
        if -opp_value > best_value:
            candidate_move = x+1
            best_value = -opp_value
        if -opp_value >= alpha:
            #print (options, x, best_value, alpha)
            break
        dummy_board[height,x] = 0

    return (best_value, candidate_move)

inf = float("inf")
def update_move(board, move, turn):
    board[np.where(board[:,move]==0)[0][-1], move] = turn
    return None
    
def unupdate_move(board, move):
    if 0 in board[:,move]:
        board[np.where(board[:,move]==0)[0][-1]+1, move] = 0
    else:
        board[0,move]=0
    return None

def n_alpha_beta_move(board, turn, depth, alpha = -inf, beta = inf, evaluation = lambda x: 0):
    dummy_board = np.copy(board) # we don't want to change the board state

    swap_player = {1:-1,-1:1} # So we can change whose turn
    options = cccc.available_moves(board) # get legal moves
    random.shuffle(options) # should inherit move order instead of randomizing


#     if len(options) == 1:
#         update_move(board,options[0])
#         if cccc.winner(dummy_board):
#             return (inf,options[0])
#         else:
#             return (0,options[0])   
    
    best_value = -inf
    
    if not options:
        print board, cccc.game_over(board)
        print 'oops, no available moves'
    cand_move = options[0]
    if depth == 0: 
        for x in options:
            update_move(dummy_board,x,turn)
            op_value = evaluation(dummy_board*swap_player[turn])

            if -op_value > best_value:
                cand_move = x
                best_value = -op_value
                alpha = max(alpha, best_value)
    #        print depth,-op_value, best_value, cand_move,alpha,beta
            if alpha >= beta:
    #                print 'pruned'
                break   #alpha-beta cutoff
            unupdate_move(dummy_board,x)
    else:
    
        for x in options:

    #        dummy_board = np.copy(board)
    #        height= np.where(board[:,x]==0)[0][-1] #connect four only
    #        dummy_board[height, x] = turn
            update_move(dummy_board,x,turn)
        
            if cccc.winner(dummy_board): #should check over and tied too
                return(inf, x)
            
            if cccc.is_full(dummy_board): #This assumes you can't lose on your turn
                return(0 , x)
            
            op_value,_ = n_alpha_beta_move( dummy_board,
                                            swap_player[turn],
                                            depth-1,
                                            alpha = - beta,
                                            beta = - alpha,
                                            evaluation = evaluation)

            if -op_value > best_value:
                cand_move = x
                best_value = -op_value
                alpha = max(alpha, best_value)
    #        print depth,-op_value, best_value, cand_move,alpha,beta
            if alpha >= beta:
    #                print 'pruned'
                break   #alpha-beta cutoff
            unupdate_move(dummy_board,x)
    #        dummy_board[height, x] = 0
    return (best_value, cand_move)

class alpha_beta:
    def __init__(self,depth):
        self.depth = depth
    def make_move(self,board,active_turn):
        #print (board,active_turn,self.depth)
        return alpha_beta_move(board,active_turn,self.depth)[1]
    
class n_alpha_beta:
    def __init__(self,depth):
        self.depth = depth
    def make_move(self,board,active_turn):
        #print (board,active_turn,self.depth)
        return n_alpha_beta_move(board,active_turn,self.depth)[1]+1

In [3]:
import sys
def progress_bar(percent, size = 20, message=''):
    percent = int(percent)
    fill = int(percent * size/100)
    sys.stdout.write('\r[' + 'X'*fill + ' '*(size-fill) + '] {:3d}% complete. '.format(percent) + message)
    sys.stdout.flush

In [246]:
games = 5
depth = 7
t0  = time.clock()
for _ in range(games):
    
    cccc.play(alpha_beta(depth),alpha_beta(depth))
    progress_bar(100.0*float(_+1)/games)
print time.clock()-t0
t0 = time.clock()
for _ in range(games):
    
    cccc.play(n_alpha_beta(depth),n_alpha_beta(depth))
    progress_bar(100.0*float(_+1)/games)
print time.clock()-t0

[XXXXXXXXXXXXXXXXXXXX] 100% complete. 994.517369659
[XXXXXXXXXXXXXXXXXXXX] 100% complete. 583.768951784


In [245]:
result = cccc.play(cccc.player(),n_alpha_beta(6))

it is X's turn, please choose a move

     |   |   |   |   |   |    
  ---------------------------        
     |   |   |   |   |   |    
  ---------------------------       
     |   |   |   |   |   |     
  ---------------------------        
     |   |   |   |   |   |    
  ---------------------------       
     |   |   |   |   |   |   
  ---------------------------       
     |   |   |   |   |   |    
  ---------------------------
   1 | 2 | 3 | 4 | 5 | 6 | 7
    
4
it is X's turn, please choose a move

     |   |   |   |   |   |    
  ---------------------------        
     |   |   |   |   |   |    
  ---------------------------       
     |   |   |   |   |   |     
  ---------------------------        
     |   |   |   |   |   |    
  ---------------------------       
     |   |   |   |   |   |   
  ---------------------------       
     |   |   | X | O |   |    
  ---------------------------
   1 | 2 | 3 | 4 | 5 | 6 | 7
    
4
it is X's turn, please choose a move

     |  

In [33]:
games  =1
for _ in range(games):
    
    cccc.play(n_alpha_beta(depth),n_alpha_beta(depth))
    progress_bar(100.0*float(_+1)/games)

pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned
pruned

In [172]:
np.all(np.ones((4,4))==1)

True

In [56]:
if np.where(np.array([1])==1)[0]:
    print True
else:
    print False

False


In [57]:
np.where(np.array([1])==1)[0]

array([0], dtype=int64)

In [60]:
if np.array([0,0]):
    print True

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [64]:
test[0,3]=4

In [65]:
test

array([[ 0.,  0.,  0.,  4.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  0.]])

In [141]:
result.p2_time

1.6000259391294094

In [95]:
unmake_move(test,3,1)

test

array([[ 1.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [96]:
make_move(test,3,-1)
test

array([[ 1.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  0.,  1.,  1.,  1.],
       [ 1.,  1.,  1., -1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [222]:
test = np.random.randint(-1,2,size=(6,7))

rows = test[:,:4]+test[:,1:5]+test[:,2:6]+test[:,3:7]
columns = test[:3,:]+test[1:4,:]+test[2:5,:]+test[3:6,:]
diag = test[:3,:4]+test[1:4,1:5]+test[2:5,2:6]+test[3:6,3:7]
rdiag = test[5:2:-1,:4]+test[4:1:-1,1:5]+test[3:0:-1,2:6]+test[2::-1,3:7]

In [223]:
print test
print rows
print columns
print diag
print rdiag

[[-1  0  0  0  1 -1 -1]
 [ 1 -1  0  0  1  1 -1]
 [-1  0  1  1  1 -1 -1]
 [ 1  0 -1  0  1  1  1]
 [ 0 -1 -1  1  1  0 -1]
 [ 1 -1  1  0  1 -1  1]]
[[-1  1  0 -1]
 [ 0  0  2  1]
 [ 1  3  2  0]
 [ 0  0  1  3]
 [-1  0  1  1]
 [ 1  1  1  1]]
[[ 0 -1  0  1  4  0 -2]
 [ 1 -2 -1  2  4  1 -2]
 [ 1 -2  0  2  4 -1  0]]
[[-1  2  2  1]
 [ 1  1  2  1]
 [-2  1  1  3]]
[[ 0 -1  2  1]
 [ 1  0  1  0]
 [ 1  2  0  1]]


In [249]:
np.any(rows == 4)

False